In [3]:
import numpy as np
import pandas as pd

In [29]:
import lib.cnn.matnpyio as io

In [4]:
import os
import re

In [5]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [6]:
#################
### base path ###
#################

# path to raw_path :  base_path/sess_no/session01/file.mat
base_path = '/media/rudy/disk2/lucy/' 

# path where the csv files are
result_path = '/home/rudy/Python2/regression_linear2/result_ridge/'

# path where to save numpy files
summary_path = '/home/rudy/Python2/regression_linear2/results/summary/'

# path where to save figure
figure_path = '/home/rudy/Python2/regresion_linear2/results/figure/'

In [15]:
# get list of session 
csv_files = os.listdir(result_path)

In [19]:
csv_files

['result_sess_no_141015channel_to_channel_all_interval_ridge.csv',
 'result_sess_no_141014channel_to_channel_all_interval_ridge.csv',
 'result_sess_no_141016channel_to_channel_all_interval_ridge.csv']

In [16]:
session =  [ re.findall('\d+', el)[0] for el in csv_files]

In [18]:
session

['141015', '141014', '141016']

In [46]:
alpha = 0.1
lowcut, highcut , order = 7, 12, 3
str_freq = 'low'+str(lowcut)+'high'+str(highcut)+'order'+str(order)        

window_size = 200
only_correct_trials = True

Cortex = ['Visual', 'Prefrontal', 'Motor', 'Parietal', 'Somatosensory'] # order matters

# les time-course vont être sorties dans cet ordre :
intervals = ['align_on_sample_from_time_-700_to_time_-500',
       'align_on_sample_from_time_-600_to_time_-400',
       'align_on_sample_from_time_-500_to_time_-300',
       'align_on_sample_from_time_-400_to_time_-200',
       'align_on_sample_from_time_-300_to_time_-100',
       'align_on_sample_from_time_-200_to_time_0',
       'align_on_sample_from_time_-100_to_time_100',
       'align_on_sample_from_time_0_to_time_200',
       'align_on_sample_from_time_100_to_time_300',
       'align_on_sample_from_time_200_to_time_400',
       'align_on_sample_from_time_300_to_time_500',
       'align_on_sample_from_time_400_to_time_600',
       'align_on_sample_from_time_500_to_time_700',
       'align_on_sample_from_time_600_to_time_800',
       'align_on_sample_from_time_700_to_time_900',
       'align_on_sample_from_time_800_to_time_1000',
       'align_on_sample_from_time_900_to_time_1100',
       'align_on_sample_from_time_1000_to_time_1200',
       'align_on_sample_from_time_1100_to_time_1300',
       'align_on_sample_from_time_1200_to_time_1400',
       'align_on_sample_from_time_1300_to_time_1500'] # must be sorted

In [ ]:
intervals = []


align_on, from_time, to_time = 'sample', -700, 500 + 1000 
window_size = 200
step = 100

for n_step in range( int( ( to_time - from_time - window_size)/step) +1 ) :
    intervals.append( 'align_on_' + align_on + '_from_time_' + str(from_time +  n_step * step) +'_to_time_'+ str(from_time +  n_step * step + window_size))
    
align_on, from_time, to_time = 'match', -500, 0 + 1900 
window_size = 200
step = 100

for n_step in range( int( ( to_time - from_time - window_size)/step) +1 ) :
    intervals.append( 'align_on_' + align_on + '_from_time_' + str(from_time +  n_step * step) +'_to_time_'+ str(from_time +  n_step * step + window_size))


In [ ]:

for sess_no in session :
    print(sess_no)
    
    #################
    # LOAD CSV FILE #
    #################
    
    # recording info path
    rinfo_path = base_path +sess_no+'/session01/' + 'recording_info.mat'
    # file path
    file_name = result_path+ 'result_sess_no_'+sess_no+'channel_to_channel_all_interval_ridge.csv'
       
    # load csv file 
    df = pd.read_csv(file_name)
    
    
    df = df[ (df['alpha']==alpha) &  (df['str_freq']==str_freq ) & (df['window_size'] == window_size) 
            & (df['only_correct_trials']==only_correct_trials) ]

    
    
    # get list of areas in the right order : sorted by cortex then sorted by area
    target_areas = []
    for cortex in Cortex : 
        areas = io.get_area_cortex(rinfo_path, cortex, unique = False)
        for area in sorted(areas) :
            target_areas.append(area)
            
    # get list of num for distinguish 2 electrodes/channels from the same area
    dico = {}
    for area in np.unique(target_areas):
        dico[area] = 0

    num_areas = []
    for i, area in enumerate(target_areas) :
        num_areas.append( dico[area] )
        dico[area] +=1
        
        
    # build matrix
    FC = np.zeros( (len(target_areas), len(target_areas), len(intervals)) )
    FC_error_bar = np.zeros( (len(target_areas), len(target_areas), len(intervals)) )
    
    
    for i, area1 in enumerate(target_areas):
        print(area1)
        for j, area2 in enumerate(target_areas):
            for t, interval in enumerate(intervals) :
                
                r2_test = df[ (df['area1'] == area1) & (df['area2'] == area2) & 
                             (df['interval'] == interval) 
                             & (df['num1'] == num_areas[i]) & (df['num2'] == num_areas[j]) ]['r2_test'].values
                
                error_bar = df[ (df['area1'] == area1) & (df['area2'] == area2) & 
                             (df['interval'] == interval) 
                             & (df['num1'] == num_areas[i]) & (df['num2'] == num_areas[j]) ]['r2_test_error_bar'].values
                
                if len(r2_test) > 0 :
                    
                    FC[i,j,t] = r2_test[0] # take the first value 
                    #FC[i,j,t] = np.mean(r2_test) # take the mean of values
                    
                    FC_error_bar[i,j,t] = error_bar[0] # take the first value
                    #FC_error_bar[i,j,t] = np.sqrt( np.sum(error_bar**2) )/len(error_bar) # take the mean of values
                    
                else:
                    print('No value for:\narea1',area1,'\narea2',area2,'\ninterval', interval)
                    
    # Create directory if not exists
    directory = summary_path +sess_no+'/'
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    ####################
    # SAVE NUMPY FILES #
    ####################
    
    
    # time course
    np.save(directory + 'FC_all_channels_low'+str(lowcut)+'high'+str(highcut)+'order'+str(order)+'_all_intervals.npy', FC)
    # error bar
    np.save(directory + 'FC_all_channels_low'+str(lowcut)+'high'+str(highcut)+'order'+str(order)+'_all_intervals_error_bar.npy', FC_error_bar)
    # label/ area names
    np.save(directory + 'label.npy', np.array(target_areas) )   
    # save time
    np.save(directory + 'time.npy', np.array(intervals))
    
#     ###########
#     # plot it #
#     ###########
    
#     for t, interval in enumerate(intervals):
        
#         fig, ax = plt.subplots(figsize=(15,15))
#         im = ax.imshow(FC[:,:,t], vmin=0)#, vmax=1.0)# cmap='jet')
#         fig.colorbar(im)
        
#         label2 = target_areas
#         label1 = target_areas

#         # We want to show all ticks...
#         ax.set_xticks(np.arange(len(label2)))
#         ax.set_yticks(np.arange(len(label1)))
#         # ... and label them with the respective list entries
#         ax.set_xticklabels(label2, linespacing=5)
#         ax.set_yticklabels(label1, linespacing=5)

#         # Rotate the tick labels and set their alignment.
#         plt.setp(ax.get_xticklabels(), rotation=0, ha="right",
#                  rotation_mode="anchor")

#         # Loop over data dimensions and create text annotations.
#         # for i in range(len(label1)):
#         #     for j in range(len(label2)):
#         #         if round(FC[i, j],3) < 0.8* FC.max() :
#         #             text = ax.text(j, i, round(FC[i, j],1),
#         #                            ha="center", va="center", color="w")
#         #         else:
#         #             text = ax.text(j, i, round(FC[i, j],1),
#         #                            ha="center", va="center", color="b")


#         ax.set_title("R_squared"+interval)
#         #fig.tight_layout()

#         # save matrix and labels as numpy file (make directory if not exists)

#         # create directory if not exists
#         directory = figure_path +sess_no+'/'
#         if not os.path.exists(directory):
#             os.makedirs(directory)


#         # save plot as jpg file
#         plt.savefig(directory+str(t)+'.FC_all_channels_low'+str(lowcut)+'high'+str(highcut)+'order'+str(order)+'_'+interval+'jpg')
#         plt.show()
    
    
    
    
    

141015
DP
DP
MT
MT
V1
V1
V1
V1
V1
V1
V1
V1
V1
V1
V1
V2
V2
V2
V2
V2
V4t
V4t
V6A
V6A
a8B
a8B
a8L
a8L
a8L
a8M
a8M
a8M
a8r
a8r
a9/46D
F1
F1
F1
F1
F2
F2
F2
F2
F2
F2
F6
F7
F7
F7
AIP


In [41]:
df.keys()

Index(['session', 'area1', 'num1', 'area2', 'num2', 'cortex1', 'cortex2',
       'interval', 'str_freq', 'window_size', 'len(ind_test)',
       'len(ind_train)', 'alpha', 'n_chans1', 'n_chans2',
       'only_correct_trials', 'r2_train', 'r2_test', 'r2_train_error_bar',
       'r2_test_error_bar', 'renorm', 'seed'],
      dtype='object')